![CoSAppLogo](images/cosapp.svg) **CoSApp** examples:

# Settings targets on output values

CoSApp systems declare a directional interface, and transform inputs into outputs.
The good practice consist in designing systems such that their inputs and outputs correcpond to the "normal", direct behaviour of the system, without presuming how it will be used in a more complex assembly, or what users will want to do with it.

As it happens, it is very common to use a given system in a reverse way, when one wishes certain outputs to reach a target value.

The first obvious way to achieve that is to setup a nonlinear problem, whereby an input is sought to satisfy an equation of the kind "target_output == target_value".

In [ ]:
from cosapp.systems import System

class SimpleFunction(System):
    def setup(self):
        self.add_inward('a', 0.0)
        self.add_inward('x', 1.0)
        self.add_outward('y', 0.0)
    
    def compute(self) -> None:
        self.y = self.x**2 - self.a


In [ ]:
from cosapp.drivers import NonLinearSolver

f = SimpleFunction('f')

solver = f.add_driver(NonLinearSolver('solver', tol=1e-9))
solver.add_unknown('x').add_equation('y == 0')

f.a = 2.0
f.run_drivers()

print(
    f"f.x = {f.x}",
    f"f.y = {f.y}",
    sep = "\n",
)

Displaying attribute `solver.problem` reveals the mathematical problem solved by the driver:

In [ ]:
solver.problem

## Set target value dynamically

In the previous example, the target value for `f.y` is hard-coded as the right-hand side of the design equation.
Changing this value requires the setup of a new mathematical problem, with a new equation.

An alternative way is to this procedure is the use of method `add_target`:

In [ ]:
from cosapp.drivers import NonLinearSolver

f = SimpleFunction('f')

solver = f.add_driver(NonLinearSolver('solver', tol=1e-9))
solver.add_unknown('x').add_target('y')

f.a = 2.0
f.y = 0.0   # set target value by setting output variable
f.run_drivers()

print(
    f"f.x = {f.x}",
    f"f.y = {f.y}",
    sep = "\n",
)

Job done!
Looking at attribute `problem` shows that the actual mathematical problem has not changed:

In [ ]:
solver.problem

The difference, however, is that the right-hand side of the equation is now dynamically set to the current value of `f.y` before each solver execution.
Therefore, we can now update the target value interactively, by simply assigning a new value to `f.y`:

In [ ]:
f.y = -0.5  # update target value dynamically

f.run_drivers()

print(
    f"f.x = {f.x}",
    f"f.y = {f.y}",
    sep = "\n",
)

As can be seen after each computation, `f.y` only reaches the targetted value *within solver tolerance*.
Indeed, if `add_target` offers a convenient way of defining target values, one must keep in mind that `f.y` remains an output, whose value is strictly determined by the actual inputs of `f`.

As a consequence, it is up to users to control the value of targetted variables *before each solver execution*.

## Targets and design methods

Targets can be particularly interesting to define [design methods](11-DesignMethods.ipynb) with a controllable target value.

In [ ]:
from cosapp.systems import System

class SimpleFunctionWithDesign(System):
    def setup(self):
        self.add_inward('a', 0.0)
        self.add_inward('x', 1.0)
        self.add_outward('y', 0.0)

        design = self.add_design_method('y')
        design.add_unknown('x').add_target('y')
    
    def compute(self) -> None:
        self.y = self.x**2 - self.a


In [ ]:
from cosapp.drivers import NonLinearSolver

f = SimpleFunctionWithDesign('f')

solver = f.add_driver(NonLinearSolver('solver', tol=1e-9))
solver.extend(f.design_methods['y'])

f.a = 2.0
f.y = -0.5
f.run_drivers()

print(
    f"f.x = {f.x}",
    f"f.y = {f.y}",
    sep = "\n",
)

Without the use of `add_target`, the only way to control the design value of `y` would be to declare an additional inward `y_target`, say, used as right-hand side value of design equation:
```python
    def setup(self):
        self.add_inward('a', 0.0)
        self.add_inward('x', 1.0)
        self.add_outward('y', 0.0)

        design = self.add_design_method('y')
        self.add_inward('y_target', 0.0)
        design.add_unknown('x').add_equation('y == y_target')
```

While this syntax works, it has the inconvenience of burdening the system with an inward only meaningful when design method `'y'` is activated.

## Pulling a targetted variable

Consider a system with a targetted output pulled at parent level.


In [ ]:
class Composite(System):
    def setup(self):
        f = self.add_child(SimpleFunction('f'), pulling='x')
        g = self.add_child(SimpleFunctionWithDesign('g'), pulling='y')

        self.connect(f.outwards, g.inwards, {'y': 'a'})  # f.y -> g.a

        # Promote design method of `g` at parent level
        # Note:
        #   g.design('y') is the same as g.design_methods['y']
        self.add_design_method('y').extend(g.design('y'))

In [ ]:
head = Composite('head')

solver = head.add_driver(NonLinearSolver('solver', tol=1e-9))

solver.extend(head.design('y'))

head.x = 3.0
head.g.y = 5.0
head.f.a = 2.0

head.run_drivers()

print(
    f"Solution:\n"
    f"head.g.x = {head.g.x}",
    f"head.g.y = {head.g.y}",
    f"\n{solver.problem!r}",
    sep = "\n",
)

What just happened?!

In this case, output `head.g.y` is pulled up at parent level. Behind the scene, an *upward* connection is created from `head.g.y` to `head.y`, such that `y` appears as a natural output of top system `head`, computed by sub-system `head.g`.

As a consequence, it seems natural to set the targetted value in the context of system `head`, that is setting `head.y` instead of `head.g.y`, when activating `head.design('y')`.
Let's try again:

In [ ]:
head = Composite('head')

solver = head.add_driver(NonLinearSolver('solver', tol=1e-9))

solver.extend(head.design('y'))  # activates a target on `head.y`

head.x = 3.0
head.y = 5.0    # set target value
head.f.a = 2.0

head.run_drivers()

print(
    f"Solution:\n"
    f"head.g.x = {head.g.x}",
    f"head.g.y = {head.g.y}",
    f"\n{solver.problem!r}",
    sep = "\n",
)

## Weak and strong targets

A target is said to be *weak* if it can be disregarded in certain situations.
By default, targets are *strong*, meaning the target equation is always enforced.

A weak target is discarded if the targetted variable
1. is an output;
2. is connected to an input.

The second condition specifically excludes pulled outputs, which is the only admissible output-output connection.

Weak targets may be useful when a targetted variable is transmitted through a chain of systems, and one wants to specify the target on the last system only.
This is typically the case in the next example, where we simulate three resistors series, and wish to determine the current between end-point voltages.


In [ ]:
from cosapp.ports import Port

class ElectricPort(Port):
    def setup(self):
        self.add_variable("I", 1.0, unit="A", desc="Current")
        self.add_variable("V", 0.0, unit="V", desc="Voltage")

class Resistor(System):
    def setup(self):
        self.add_input(ElectricPort, 'elec_in')
        self.add_output(ElectricPort, 'elec_out')

        self.add_inward("R", 1e2, unit="ohm", desc="Resistance")
        self.add_outward("deltaV", 0.0, unit="V")

        # Off-design constraint: compute current to reach target voltage
        self.add_unknown("elec_in.I").add_target("elec_out.V", weak=True)
    
    def compute(self):
        elec_in, elec_out = self.elec_in, self.elec_out
        self.deltaV = self.R * elec_in.I
        elec_out.I = elec_in.I
        elec_out.V = elec_in.V - self.deltaV

class ThreeResistorSeries(System):
    def setup(self):
        R1 = self.add_child(Resistor("R1"), pulling="elec_in")
        R2 = self.add_child(Resistor("R2"))
        R3 = self.add_child(Resistor("R3"), pulling="elec_out")
        
        self.connect(R1.elec_out, R2.elec_in)
        self.connect(R2.elec_out, R3.elec_in)


Each resistor defines an inner off-design problem, in which current is unknown, and a target is set on the output voltage.
When several resistors are connected, local unknown currents are discarded every time they belong to a connected input port, which occurs at each node point connecting adjacent resistors.
Likewise, local weak targets on voltages are discarded at each connecting node, where output voltage is transmitted to the next resistor.

Overall, one unknown (the incoming current into the first resistor, pulled as `elec_in.I`) and one target (the output voltage of the last resistor, pulled as `elec_out.V`) remain.


In [ ]:
s = ThreeResistorSeries('s')

s.R1.R = 100
s.R2.R = 50
s.R3.R = 250

s.elec_in.V = 10
s.elec_out.V = -2

# Set bogus target values at connection points
# These values will be discarded, as targets are weak
s.R1.elec_out.V = 1.23e4
s.R2.elec_out.V = -8e17

s.add_driver(NonLinearSolver('solver'))
s.run_drivers()

voltages = [s.elec_in.V]
voltages.extend(s[f"R{i}.elec_out.V"] for i in range(1, 4))

print(
    f"Solution:\n"
    f"s.elec_in.I = {s.elec_in.I}",
    f"s.elec_in.V = {s.elec_in.V}",
    f"s.elec_out.V = {s.elec_out.V}",
    f"Voltages = {voltages}",
    f"\nOverall resistance: {(s.elec_in.V - s.elec_out.V) / s.elec_in.I :.2f} Ohm",
    sep = "\n",
)


In [ ]:
s.drivers['solver'].problem

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.DataFrame.from_dict(
    {
        "V": voltages,
        "index": list(range(len(voltages))),
        "node": list('ABCD'),
    }
)

fig = px.line(df, x="node", y="V",
    title = "Voltage profile",
)
fig.update_layout(
    height=450,
    hovermode='x',
)
fig.show()

## Targetted expressions

A target can also be set on an evaluable expression, such as `norm(v)` or `y * (y + 1)`, as long as the expression only involves a single variable.

In [ ]:
class CubicFunction(System):
    def setup(self):
        self.add_inward('x', 1.0)
        self.add_outward('y', 0.0)
    
    def compute(self) -> None:
        self.y = self.x**3

f = CubicFunction('f')

solver = f.add_driver(NonLinearSolver('solver', tol=1e-9))

solver.add_unknown('x').add_target('abs(y)')

f.y = 8  # value used for targetted expression
f.x = 4  # positive initial value of unknown
f.run_drivers()

print(
    f"f.x = {f.x}",
    f"f.y = {f.y}",
    sep = "\n",
)

Same computation, with a different initial guess on `f.x`:

In [ ]:
f.y = 8
f.x = -5  # negative initial value
f.run_drivers()

print(
    f"f.x = {f.x}",
    f"f.y = {f.y}",
    sep = "\n",
)

In [ ]:
import logging

f = CubicFunction('f')

solver = f.add_driver(NonLinearSolver('solver', tol=1e-9))

try:
    solver.add_unknown('x').add_target('2 * y + cos(pi * x)')
except Exception as error:
    logging.exception(error)
